# Combining the 2 largest Movies Datasets
### Data Engineering Capstone Project

#### Project Summary
IMDB and TMDB are the top 2 competitors when it comes to movie and television metadata [(Basu, 2018)](https://www.makeuseof.com/tag/best-alternative-imdb/). The aim of this project is to combine the datasets from these 2 sources. The ETL pipeline established will generate a combined database, which allows users to effectively query, analyze and utilize both major datasets, rather than having to perform seperate analysis on both. 

The report follows these steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

The ETL pipeline created follows these steps:
* Step 1: Upload dataset source files to S3
* Step 2: Use Spark to clean source files
* Step 3: Output Spark dataframe as `.parquet` files back to S3
* Step 4: Create data model on Redshift
* Step 5: Load `.parquet` files from S3 to Redshift
* Step 6: Perform data quality checks

### Step 1: Scope the Project and Gather Data

#### Scope 

The objective of this project is to create a large data model involving movie data. The data model will be created by 2 seperate sources of movie data, IMDB and TMDB. With this data model, analysts can work to analyze movie data more efficently. 

For example, TMDB has data regarding the revenue and budget of a movie whereas the IMDB dataset has information regarding specific actors. One use cases would be to identify actors who tend to act in higher budget or higher revenue movies.

By combining both datasets, we are able to get the best of both worlds.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

The 2 datasets are obtained from IMDB and TMDB. The IMDB dataset came directly from IMDB itself, via [here](https://datasets.imdbws.com/). The TMDB dataset is scrapped and available on Kaggle [here](https://www.kaggle.com/rounakbanik/the-movies-dataset). 

To see the exact rows of the dataset, please view the exploration notebook [here](https://github.com/AhmadHatziq/data-engineering-nanodegree/blob/main/capstone-project/workings/exploration_and_cleaning.ipynb). 

#### IMDB Dataset
There are 6 files in total, all available as `.tsv` files. The file details are as follows:
- title-principals.tsv: 42,750,666 rows. Contains the principal cast/crew for titles.
- title-akas.tsv: 24,978,357 rows. Contains region, language and aliases for the titles.
- title-basics.tsv: 7,554,298 rows. Contains title runtimes, years and genres.  
- title-crew.tsv: 7,554,298 rows. Contains directors and writers of the titles.
- title-episode.tsv: 54,71,901 rows. Contains the episode information for relevant titles.
- title-ratings.tsv: 1,116,071 rows. Contains the IMDb rating and votes information for titles.
- name-basics.tsv: 10,671,837 rows. Contains the information for all people involved in the titles.

#### TMDB Dataset
There are 5 files in total, all available as `.csv` files. The file details are as follows:
- movies_metadata.csv: 45,363 rows. Represents the main metadata of TMDB movies.
- keywords.csv: 46,419 rows. Contains the movie plot keywords.
- credits.csv: 30,457 rows. Consists cast and crew details for the movies.
- links.csv: 45,843 rows. Contains the TMDB and IMDB IDs of all the movies in this TMDB dataset.
- ratings.csv: 26,024,289 rows. Contains the ratings of TMDB users.

### Step 2: Explore and Assess the Data

As there are a total of 11 tables, please head to this [notebook](https://github.com/AhmadHatziq/data-engineering-nanodegree/blob/main/capstone-project/workings/exploration_and_cleaning.ipynb) to view the data exploration and data cleaning.

#### Explore the Data 

The IMDB dataset is generally clean, as it comes directly from IMDB. However, there are some discrepancies with the TMDB dataset. This could be becuase the dataset is obtained via web-scrapping. 

For example, the file `keywords.csv` is made of 2 columns, `id` and `keywords`. `id` is a unique integer however, some rows have empty or invalid `id` values. These invalid rows are subsequently dropped.

#### Cleaning Steps

As the column names were unintuitive, better column names were used. Please check the data dictionary for the data definitions.

In the original dataset, the columns were all identified as type `string`. Hence, exploratory analysis was conducted to ascertain the correct variable types. 

In the IMDB dataset, the string `\N` was used to define null. Hence, this string was replaced with `null` values throughout the dataset.

For columns of type `string`, the length of the longest value was recorded. This will help when the data model is defined on Redshift later. For instance, for the column `TITLE_ID`, the longest record was 10 characters long. Hence, a variable of `VARCHAR(15)` was assigned. Additional character width was given as buffer.

#### Cleaning with Spark
The `.tsv` and `.csv` files were uploaded to S3. These files were then cleaned using Spark within the Udacity workspace. Following which, the Spark dataframes were exported as `.parquet` files back to S3.

Cleaning the data using Spark has made the ETL process much more efficent. This is as the dataset files were reduced from 5.3GB to 1.5GB. This will make the loading of data into the database a lighter workload.

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model


<img src="images/imdb_schema.png">
IMDB Schema

<br>

<img src="images/tmdb_schema.png">

TMDB Schema

Each dataset has their own respective STAR schemas. 

As the objective of the project is to create a combined dataset, the final schema will combine both schemas, using the `KAGGLE_LINKS` table as a link between IMDB and TMDB. 

<img src = "images/data_model.png">
            
Final Schema    

#### Schema justification

The IMDB dataset (total 100,097,428 records) has way more records than the TMDB dataset (total 26,192,371 records). Hence, the IMDB schemas has a more 'flat' star schema. This is so that the joins will be less heavy and access less data. In contrast, the TMDB schema is less 'flat'. Most of the rows are located within the main metadata table. This is done so that the TMDB metadata table can be joined directly with any of the IMDB tables. 

However, note that the KAGGLE_LINKS table is used to join with the IMDB data, rather than linking KAGGLE_MOVIES_METADATA directly with the IMDB tables. We use the smaller KAGGLE_LINKS table for any joins with the IMDB dataset. This allows joins to be much quicker. Furthermore, if we had multiple users trying to join the IMDB tables with the large KAGGLE_MOVIES_METADATA, this will cause a strain on the Redshift cluster. 

The final justification is that we are able to preserve the most amount of rows from the source data. This allows the analyst / user to analyze all the data, and then deciding which rows are to be discarded. We have to acomodate for the KAGGLE_TMDB tables as they are coming directly from web scraping.

Perhaps in the future, data quality checks can be implemented, to ensure that the scrapped data is within a certain format. This can only be done after discussions with all the business stakeholders have been completed.

#### 3.2 Mapping Out Data Pipelines
The ETL pipeline is created via the following steps:
* Step 1: Upload dataset source files to S3
* Step 2: Use Spark to clean source files
* Step 3: Output Spark dataframe as `.parquet` files back to S3
* Step 4: Create data model on Redshift
* Step 5: Load `.parquet` files from S3 to Redshift
* Step 6: Perform data quality checks

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

The relevant files used in the ETL process are: 
- `create_cluster.py`: IaC file. Used to create the Redshift database and store login credentials. 
- `delete_cluster.py`: IaC file. Used to delete the Redshift database. 
- `dwh.cfg`: Configuration file used for storing AWS and Redshift credentials.
- `source_file_to_parquet.py`: Loads the raw data files from S3 into Spark, performs data cleaning and writes parquet files back into S3.
- `parquet_to_redshift.py`: Loads the parquet files from S3 into the Redshift database.
- `sql_statements.py`: SQL statements used in the ETL process.
- `data_quality.py`: Performs data quality checks.
- `etl.py`: Executes the whole ETL process, using all the above files except for `create_cluster.py` and `delete_cluster.py`.

The pipeline can be executed by running `python etl.py` on the commandline. 

Relevant information is logged to indicate the ETL progress. After successful execution, the following output should be observed: 

<img src = "images/success_output.png">

Using the Udacity workspace, the whole process took 1465 seconds ≈ 24 minutes

#### 4.2 Data Quality Checks

The data quality checks can be found in this [notebook](https://github.com/AhmadHatziq/data-engineering-nanodegree/blob/main/capstone-project/workings/data_quality.ipynb). 

There are 2 data quality checks: 
1. Row count of all tables
2. Ensuring acceptable error threshold for missing IMDB TITLE IDs.

#### 4.2.1. Row Count Check

```python
for table in table_names:
    sql_string = "SELECT COUNT(*) FROM " + table + ";"
    cur.execute(sql_string)
    conn.commit()
    row = cur.fetchone()
    num_rows = row[0]

    if num_rows <= 0:
        raise ValueError('Data Quality ERROR: table ' + table + ' has 0 rows.')
            
```

The above data quality check ensures that the ETL process are correctly executed. Each table should have at least 1 row in the end. 

#### 4.2.2. Error Threshold for Missing TITLE IDs

```python
    # Get number of non_existing IDs
    query = "SELECT COUNT(*) FROM KAGGLE_LINKS WHERE KAGGLE_LINKS.IMDB_ID NOT IN (SELECT IMDB_TITLE_PRINCIPALS.TITLE_ID FROM IMDB_TITLE_PRINCIPALS );"
    cur.execute(query)
    conn.commit()
    row = cur.fetchone()
    missing_imdb_ids = row[0]

    # Get total counts of KAGGLE_LINKS table
    query = "SELECT COUNT(*) FROM KAGGLE_LINKS"
    cur.execute(query)
    conn.commit()
    row = cur.fetchone()
    kaggle_imdb_counts = row[0]
    
    # Checks if error proportion is tolerable
    missing_proportion = missing_imdb_ids / kaggle_imdb_counts

    if missing_proportion >= (0.002 * kaggle_imdb_counts):
        raise ValueError("Error threshold exceeded, please check data quality of source files.")
            
```

The purpose of the pipeline is to consolidate the information between the IMDB and TMDB databases.

The common attribute that they share is the IMDB TITLE ID. The table KAGGLE_LINK links the 2 databases together.

We tolerate an error margin as the KAGGLE TMDB dataset is obtained via web-scrapping, which is prone to errorneous data entries.

This data quality operation checks if the number of TITLE_IDs within TMDB, that does not exist within IMDB, is within an error threshold.

The error margin is currently set at 0.2% x TOTAL_RECORDS_IN_KAGGLE_LINK_TABLE.

With the current dataset, an error rate of 0.0016578321663067426 ≈ 0.1658% is observed
        

#### 4.3 Data dictionary 

Data dictiomary can be found [here](https://github.com/AhmadHatziq/data-engineering-nanodegree/blob/main/capstone-project/data_dictionary.md). 

#### Step 5: Complete Project Write Up


#### Rationale for choice of tools and technologies

S3 is chosen to store the source files and parquet files as it is a cheap, easy to use and scalable. 
Furthermore, S3 is more scalable and durable than HDFS. S3 is always persistent than HDFS. However, HDFS should be chosen if performance is an issue [(Parker, 2019)](https://www.xplenty.com/blog/storing-apache-hadoop-data-cloud-hdfs-vs-s3/)

Spark is chosen as it is able to efficently process large datasets.

Parquet is chosen as its columnar storage allows us to reduce the dataset size from 5.3GB to 1.5GB.

Redshift is chosen as the database as it can natively load parquet files directly from S3, is easy to use and scalable.

#### How often should the data be updated?

As the ETL process takes approximately 30 minutes for a TRUNCATE LOAD operation, the database should be updated once per day. This is as generally, the rate of release of movies is not that fast.

#### What happens if the data was increased by 100x?

If the data is increased by 100x, the ETL process will take longer. Instead of TRUNCATE LOAD, we have to do INCREMENTAL LOAD. We need to identify the new rows and append them to the database. 

#### What happens if the data is used in a dashboard that must be updated on a daily basis by 7am every day?

Given the current runtime of the ETL process takes approximately 30 minutes, we can schedule the ETL to occur at 6am everyday. The extra 30 minutes can be used as a buffer in case errors occur.

#### What happens if the database needs to be accessed by 100+ people?

We can create data marts to ensure that the analytical workloads used by the 100+ people are fast and does not bring the database down.

Additionally, we can create specific Cassandra tables if the users work are always revolving around the same queries.